In [ ]:
%matplotlib inline

import os
from os.path import join as pjoin
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
gateFile = "C:/WorkSpace/data/tcha/gates.shp"
trackFile = "C:/WorkSpace/data/tcha/tracks/tracks.00003_line.shp"

In [ ]:
gates = gpd.read_file(gateFile)
gates['count'] = 0
gates['meanlfintensity'] = np.nan
gates['minlfintensity'] = np.nan
gates['cat1'] = 0
gates['cat2'] = 0
gates['cat3'] = 0
gates['cat4'] = 0
gates['cat5'] = 0
trackDF = gpd.read_file(trackFile)
# Category is based on central pressure. This equally could be maximum wind speed
trackDF['category'] = pd.cut(trackDF['pCentre'], 
                             bins=[0, 930, 955, 970, 985, 990, 1020], 
                             labels=[5,4,3,2,1,0])

In [ ]:
for i, gate in enumerate(gates.itertuples(index=False)):
    ncrossings = 0
    crossings = trackDF.crosses(gate.geometry)
    ncrossings = np.sum(trackDF.crosses(gate.geometry))
    if ncrossings > 0:
        gates['count'].iloc[i] = ncrossings
        gates['meanlfintensity'].iloc[i] = trackDF['pCentre'][crossings].mean()
        gates['minlfintensity'].iloc[i] = trackDF['pCentre'][crossings].min()
        cathist, bins = np.histogram(trackDF['category'][crossings], bins=[0,1,2,3,4,5, 6])
        gates['cat1'].iloc[i] = cathist[1]
        gates['cat2'].iloc[i] = cathist[2]
        gates['cat3'].iloc[i] = cathist[3]
        gates['cat4'].iloc[i] = cathist[4]
        gates['cat5'].iloc[i] = cathist[5]
    else:
        gates['meanlfintensity'].iloc[i] = np.nan
        gates['minlfintensity'].iloc[i] = np.nan
        gates['cat1'].iloc[i] = 0
        gates['cat2'].iloc[i] = 0
        gates['cat3'].iloc[i] = 0
        gates['cat4'].iloc[i] = 0
        gates['cat5'].iloc[i] = 0
        
        

In [ ]:
gates

In [ ]:
width=0.4
fig, ax = plt.subplots(3,1,figsize=(12,16),sharex=True)
ax[0].bar(gates['gate']-0.5*width, gates['cat5']+gates['cat4']+gates['cat3'],width=width, color='r', label='Severe TCs')
ax[0].bar(gates['gate']+0.5*width, gates['count'],width=width, color='b', label='TCs')
ax[0].legend()
ax[1].plot(gates['gate'], gates['minlfintensity'], label='Minimum landfall intensity')
ax[1].plot(gates['gate'], gates['meanlfintensity'], color='r', label='Mean landfall intensity')
ax[1].legend(loc=2)
ax[1].set_ylim((850, 1020))
ax[2].bar(gates['gate'], gates['count'])
ax[2].set_xlim((0,48))
ax[2].set_xticks(np.arange(0,48,2))
ax[2].set_yticks(np.arange(0,24,2))
ax[2].set_xticklabels(gates['label'][::2], rotation='vertical')

In [ ]:
print("Number of events:", pd.unique(trackDF['CycloneNumb']).size)
print("Number of landfalls:", gates['count'].sum())